# Measurement Error Mitigation for Local Noise Model

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

Crosstalk noise is a kind of complex noise source in the measurement process. It is of great significance to describe crosstalk noise efficiently to improve the slow-release accuracy of measurement results. This tutorial will introduce a theory that can efficiently characterize and mitigate crosstalk noise and its application on [Baidu quantum platform]( https://quantum.baidu.com/).

## Theory

We have already introduced that measuring noise mitigation can be divided into two processes. The first step is calibration. By preparing calibration circuit and constructing calibration matrix, noise information can be obtained. This step can be carried out by **complete model calibration** and **tensor product calibration**; The second step is mitigation. Through the obtained calibration matrix, we can mitigate the noise of the measurement results. There are many methods in this step, such as **matrix inversion method** [1], **least square method**, **iterative Bayesian unfolding (IBU) method** [2], **Neumann method** [3], etc.

### Local Noise Calibration

The above **complete model calibration** consumes a lot of resources, and **tensor product calibration** requires no crosstalk (this is not an easy condition to meet in the experiment). IBM proposes a method for modeling measurement noise based on continuous time Markov process (CTMP)[4]. Compared with **tensor product calibration**, this method can better describe crosstalk noise and save a lot of resources compared with **unstructured calibration**. The CTMP method considers that the calibration matrix $R$ can be regarded as the solution of a differential equation

$$
\begin{align}
\dot{R}(t) = GR(t)
\end{align}
$$

with the initial value of $R(0) = I$. At this time, $G$ is the generator of a continuous time Markov process, $\langle y\vert G \vert x\rangle$ represents the probability of transition from state $x$ to state $y$ and $y\neq x$. By solving the differential equation, we can get the expression of the calibration matrix

$$
\begin{align}
R = e^G.
\end{align}
$$

This means that the calibration matrix $R$ is uniquely determined by the generation matrix $G$ and the expression of $G$ is

$$
\begin{align}
G = \sum_{i}λ_i G_i,
\end{align}
$$

Among them: $λ_i$ is expressed as the error probability, obtained from the calibration data on the experiment, $G_i$ is used to describe the generator corresponding to the error and is a value that can be directly calculated theoretically.

$λ_i$ characterizes the crosstalk noise. Before discusses how to determine $λ_i$, we give the definition of crosstalk noise. The crosstalk noise is related to the number of bits. When the input ground state is prepared and measured for a given $k$ qubits, the output state is completely opposite to the input state (that is, $k$ qubits are all inverted), then we say that there is a $k$ order crosstalk noise between these $k$ qubits. Intuitively, the $k$ order crosstalk noise means that the $k$ qubits are fully correlated. Suppose we want to characterize the $k$ order crosstalk noise of $S = \left\{q_0,..., q_{k-1}\right\}$ qubits, we first calculate $2^k × 2^k$ calibration matrix $R^S$ from the calibration data set based on the complete model, then the $2^k$ inverse diagonal elements of this matrix describe the probability that the output states of $k$ bits are completely opposite to the input states, that is, the probability that all $k$ qubits are flipped. We find the logarithm of the matrix for $R^S$, and the $2^k$ inverse diagonal elements of the resulting matrix are defined as the $k$ order crosstalk noise weight coefficient $\left\{ λ_ i \right\}_{i = 0}^{2^k-1}$ of $S$, which completely characterize the $k$ order crosstalk noise of $k$ qubits represented by $S$:

$$
\begin{align}
\left\{λ_i \right\}_{i=0}^{2^k-1}=antidiag \left(\log\left(R^S \right)\right),
\end{align}
$$

Where $antidiag(R)$ represents the selection of all the inverse diagonal elements of the matrix $R$ (the selection order is from the lower left corner to the upper right corner), and $log(R)$ represents the logarithmic operation of the matrix. According to the definition of weight coefficient, the crosstalk noise of order $k$ is described by $2^k$ elements, and each weight coefficient $λ_i$ characterizes a specific $k$ qubit flip error which is described by a $2^n×2^n$ generator matrix $G_i$:

$$
\begin{align}
G_i  = |2^n-1 \oplus i⟩⟨i|-|i⟩⟨i|,
\end{align}
$$

where $\oplus$ represents binary exclusive or operation, $2^n-1 \oplus i$ represents binary expansion of positive integer $i$ and bitwise negation. Intuitively, $G_i$ describes the event: "the input standard base quantum state $|i⟩$, the output is either the bit string $i$, or the completely inverted bit string $2^n-1 \oplus i$". It should be emphasized here that $G_i$ default tensor to the entire $n$ qubit space, thus $2^n × 2^n$. Exhausting $i \in \left\{0,..., 2^k-1 \right\}$, we get $2^k$ generators in total.

The experiment shows that when two qubits are not interconnected and far apart, the crosstalk between these two bits is very weak. Therefore, we can use this property to select which $k$ qubits in the quantum measurement device need to consider the $k$ order crosstalk noise in combination with the topology of the hardware itself. Generally speaking, the hardware topology with $n$ qubits can be described by using a simple undirected graph $Ω = (V, E)$ where $V$ is the node set (corresponding to the qubits in the hardware), and $E$ is the edge set (describing the interconnection of the qubits in the hardware). We assume that there is $k$ order crosstalk noise between some $k$ qubits in hardware if and only if the node set corresponding to the $k$ qubits is a $k$ order connected subgraph in a simple undirected graph. Intuitively, if the nodes corresponding to these $k$ qubits are connected to each other on the undirected graph, we think that there will be $k $order crosstalk noise between them.

In fact, it is not difficult to find that **complete model and tensor product model are two special cases of local noise model**. When we consider only $k=1$ order crosstalk noise, it becomes tensor product model. When we consider $k=n$ order crosstalk noise, it is the complete model.

### Local Noise Mitigation

When we use the **local noise calibration** method to obtain the generation matrix $G$, we can expand the inverse of the calibration matrix $R^{-1}$ so that we can mitigate noisy measurement outcomes. Let

$$
\begin{equation}
    \gamma = \max_{x\in\left\{0, 1\right\}^n}{-\langle x\vert G\vert x \rangle}.
\end{equation}
$$

Define

$$
\begin{equation}
    B = I + \gamma^{-1}G,
\end{equation}
$$

we can have

$$
\begin{equation}
    R^{-1} = e^{-G} = e^{\gamma} \cdot e^{-\gamma B} = \sum_{\alpha=0}^{\infty}\frac{e^{\gamma}\left(-\gamma\right)^\alpha}{\alpha!}B^{\alpha}.
\end{equation}
$$

The value of $\alpha$ can be truncated by the distance between two iterations being less than a certain threshold.

## Practice

### Task description

We show the measurement error mitigation through a GHZ state.
We first generate a large number of GHZ states of $2$ qubits (strictly speaking, it should be called Bell state. For convenience of description, it is unified as GHZ state in the tutorial), measure them on the $Z$-basis, obtain the statistical information of the output states, and then calculate the expectation value of observation operator

$$
\begin{equation}
O = {\vert0\rangle\langle0\vert}^{\otimes2}+{\vert1\rangle\langle1\vert}^{\otimes2}.
\end{equation}
$$

Ideally, the result should be 1, but if there is noise, the expected value will not be equal to $1$.

### Noisy results

First, we import the necessary libraries

In [ ]:
import copy
import numpy as np
import functools
import networkx as nx
import matplotlib.pyplot as plt
import seaborn
import pandas
from QCompute import *

from Extensions.QuantumErrorProcessing.qcompute_qep.measurement.correction import CTMPCorrector
from Extensions.QuantumErrorProcessing.qcompute_qep.measurement.utils import plot_histograms, dict2vector
from Extensions.QuantumErrorProcessing.qcompute_qep.utils import expval_from_counts
from Extensions.QuantumErrorProcessing.qcompute_qep.utils.types import get_qc_name
from Extensions.QuantumErrorProcessing.qcompute_qep.utils.circuit import execute

Then, we need to obtain the token from the Quantum Leaf to use the real quantum computing machine

In [ ]:
# Please log in the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
Define.hubToken = "Token"

# Set the default maximal number of measurement shots
MAX_SHOTS = 8192

Then we begin to construct the quantum circuit corresponding to the GHZ state, and obtain the statistical results of the output state through the measurement in the $Z$ direction

In [ ]:
qp = QEnv()
n = 2
qp.Q.createList(n)
H(qp.Q[0])
for i in range(1, n):
    CX(qp.Q[0], qp.Q[i])

MeasureZ(*qp.Q.toListPair())

Now we construct the observation operator

In [ ]:
proj0 = np.array([[1, 0], [0, 0]]).astype(complex)
proj1 = np.array([[0, 0], [0, 1]]).astype(complex)
O = functools.reduce(np.kron, [proj0] * n) + functools.reduce(np.kron, [proj1] * n)

Select the quantum computer to be used for calculation. LocalBaiduSim2 is the ideal simulator, and CloudBaiduQPUQian is the real quantum machine provided by Baidu Quantum

In [ ]:
ideal_qc = BackendName.LocalBaiduSim2

noisy_qc = BackendName.CloudBaiduQPUQian

noisy_qc_name = get_qc_name(noisy_qc)

Obtain the execution result of the quantum computer (that is the statistical information of the output state), and use the statistical information to calculate the expected value of the observation operator $O$

In [ ]:
# WARNING! We must deep copy the quantum program since `execute` will destroy the quantum program

# Ideal case.
counts_ideal = execute(qp=copy.deepcopy(qp), qc=ideal_qc, shots=MAX_SHOTS)
val_ideal = expval_from_counts(O, counts_ideal)

print("The ideal expectation value is: {}".format(val_ideal))

# Noisy case.
counts_noisy = execute(qp=copy.deepcopy(qp), qc=noisy_qc, shots=MAX_SHOTS)
val_noisy = expval_from_counts(O, counts_noisy)

print("The noisy expectation value is: {}".format(val_noisy))

### Corrected results

After the calculation results are obtained, we can correct the noise situation. As mentioned above, there are two steps in measuring noise mitigation: calibration + correction. It is worth mentioning that we correct the statistical results of the output state, and then use the corrected statistical results to calculate the expected value. Next, the effects of the local noise model are shown one by one. We need to emphasize that for local noise correction, only local noise calibration should be selected. At the same time, we must also consider the topological structure of quantum computer. In order to facilitate users' use, we have built in the topology of common quantum computers. If it is not included in the built-in topology, users need to use Networkx to set the topology by themselves. In this tutorial, the topological structure of the quantum computer we selected has been built in. Here, $k$ represents the crosstalk noise order, and the default value is $2$.

In [ ]:
corr_ctmp = CTMPCorrector(qc=noisy_qc, calibrator='ctmp', qubits=range(n), k=2)

# If the topology of the quantum computer is not built in, users should set the topology by themselves.
# G = nx.Graph()
# G.add_nodes_from([0, 1, 2, 3, 4])
# G.add_edges_from([(0, 1), (1, 2), (1, 3), (3, 4)])
# corr_ctmp = CTMPCorrector(qc=noisy_qc, calibrator='ctmp', qubits=range(n), topo=G)

counts_ctmp = corr_ctmp.correct(counts_noisy)

# Compute the expectation value from corrected counts
val_ctmp = expval_from_counts(O, counts_ctmp)

print("The 'CTMP Calibrator + CTMP Corrector' "
      "mitigated expectation value is: {}".format(val_ctmp))

## Summary
In our tutorial, we discussed the case of $2$ order crosstalk noise. In the local noise model, combined with the topological structure of quantum computer, we only used $XX $calibration circuits. Compared with the complete calibration model, it saves resources and ensures accuracy. Compared with the tensor product model, it can more accurately describe the measurement noise, which shows the superiority of the local noise model.

## References

[1] Wikipedia contributors. "Moore–Penrose inverse." [Wikipedia](https://en.wikipedia.org/w/index.php?title=Moore%E2%80%93Penrose_inverse&oldid=1022461613), 2021.

[2] Nachman, Benjamin, et al. "Unfolding quantum computer readout noise." [npj Quantum Information](https://arxiv.org/abs/1910.01969) 6.1 (2020): 1-7.

[3] Wang, Kun, Yu-Ao Chen, and Xin Wang. "Measurement Error Mitigation via Truncated Neumann Series." [arXiv](https://arxiv.org/abs/2103.13856) preprint arXiv:2103.13856, 2021.

[4] Bravyi, Sergey, et al. "Mitigating measurement errors in multiqubit experiments." [Physical Review A](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.042605) 103.4 (2021): 042605.